In [1]:
import numpy as np
from scipy import sparse
from collections import defaultdict
import heapq
import time
import memory_profiler
import random

In [2]:
class SimpleInteractionTracker:
    def __init__(self, n_users):
        self.interactions = defaultdict(lambda: defaultdict(int))
        self.n_users = n_users
    
    def record_group_interaction(self, user_ids):
        for i in range(len(user_ids)):
            for j in range(i + 1, len(user_ids)):
                user1, user2 = user_ids[i], user_ids[j]
                self.interactions[user1][user2] += 1
                self.interactions[user2][user1] += 1
    
    def get_top_interactions(self, user_id, k=3):
        if user_id not in self.interactions:
            return []
        return heapq.nlargest(k, self.interactions[user_id].items(), key=lambda x: x[1])

In [3]:
class EfficientInteractionTracker:
    def __init__(self, n_users):
        self.n_users = n_users
        # Initialize using CSR format (better for row operations)
        self.interactions = sparse.csr_matrix((n_users, n_users), dtype=np.int32)
    
    def record_group_interaction(self, user_ids):
        # Convert to lil format for efficient matrix updates
        if not isinstance(self.interactions, sparse.lil_matrix):
            self.interactions = self.interactions.tolil()
        
        for i in range(len(user_ids)):
            for j in range(i + 1, len(user_ids)):
                user1, user2 = user_ids[i], user_ids[j]
                self.interactions[user1, user2] += 1
                self.interactions[user2, user1] += 1
    
    def get_top_interactions(self, user_id, k=3):
        # Convert to CSR format for efficient row operations
        if not isinstance(self.interactions, sparse.csr_matrix):
            self.interactions = self.interactions.tocsr()
        
        row = self.interactions.getrow(user_id).tocoo()
        interactions = list(zip(row.col, row.data))
        return heapq.nlargest(k, interactions, key=lambda x: x[1])

In [4]:
def generate_random_interactions(n_users, n_groups, group_size):
    interactions = []
    for _ in range(n_groups):
        group = random.sample(range(n_users), group_size)
        interactions.append(group)
    return interactions


def benchmark_implementations(n_users, n_groups, group_size):
    # Generate random interaction data
    print(f"Generating {n_groups} random interactions for {n_users} users...")
    interactions = generate_random_interactions(n_users, n_groups, group_size)
    
    implementations = {
        "Simple": SimpleInteractionTracker(n_users),
        "Efficient": EfficientInteractionTracker(n_users)
    }
    
    results = {}
    
    for name, impl in implementations.items():
        print(f"Benchmarking {name} implementation")
        
        # Measure insertion time
        start_time = time.time()
        for group in interactions:
            impl.record_group_interaction(group)
        insertion_time = time.time() - start_time
        
        # Measure query time (average of 100 random queries)
        query_times = []
        for _ in range(100):
            user_id = random.randint(0, n_users-1)
            start_time = time.time()
            impl.get_top_interactions(user_id)
            query_times.append(time.time() - start_time)
        avg_query_time = sum(query_times) / len(query_times)
        
        # Measure memory usage
        if isinstance(impl, SimpleInteractionTracker):
            memory = sum(len(v) for v in impl.interactions.values()) * 12  # Approximate
        else:
            memory = impl.interactions.data.nbytes + impl.interactions.indptr.nbytes + impl.interactions.indices.nbytes
        
        results[name] = {
            "insertion_time": insertion_time,
            "avg_query_time": avg_query_time,
            "memory_bytes": memory
        }
    return results

In [5]:
# Test with different scales
scales = [
    (10000, 5000, 5),
    (10000, 5000, 5),
    (10000, 5000, 10),
    (100000, 50000, 5),
    (100000, 50000, 10),
    (100000, 100000, 10),
]

for n_users, n_groups, group_size in scales:
    print(f"\n{'='*50}")
    print(f"Testing with {n_users} users, {n_groups} groups, size {group_size}")
    print('='*50)
    
    results = benchmark_implementations(n_users, n_groups, group_size)
    
    # Compare implementations
    print("\nComparison Summary:")
    for metric in ["insertion_time", "avg_query_time", "memory_bytes"]:
        print(f"\n{metric}:")
        sorted_impls = sorted(results.items(), key=lambda x: x[1][metric])
        for name, result in sorted_impls:
            value = result[metric]
            if metric == "memory_bytes":
                print(f"{name}: {value/1024/1024:.2f} MB")
            else:
                print(f"{name}: {value:.6f} seconds")


Testing with 10000 users, 5000 groups, size 5
Generating 5000 random interactions for 10000 users...
Benchmarking Simple implementation
Benchmarking Efficient implementation

Comparison Summary:

insertion_time:
Simple: 0.018325 seconds
Efficient: 0.205732 seconds

avg_query_time:
Simple: 0.000001 seconds
Efficient: 0.000065 seconds

memory_bytes:
Efficient: 0.80 MB
Simple: 1.14 MB

Testing with 10000 users, 5000 groups, size 5
Generating 5000 random interactions for 10000 users...
Benchmarking Simple implementation
Benchmarking Efficient implementation

Comparison Summary:

insertion_time:
Simple: 0.017093 seconds
Efficient: 0.185469 seconds

avg_query_time:
Simple: 0.000001 seconds
Efficient: 0.000089 seconds

memory_bytes:
Efficient: 0.80 MB
Simple: 1.14 MB

Testing with 10000 users, 5000 groups, size 10
Generating 5000 random interactions for 10000 users...
Benchmarking Simple implementation
Benchmarking Efficient implementation

Comparison Summary:

insertion_time:
Simple: 0.0679